In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


!pip install gdown
!gdown "https://drive.google.com/uc?id=1w09SlKhsfw8wkY7Z0z7KbRmPEDekpmT3"

# Define the file path, column names to remove keep, and number of rows to sample
file_path = "CABS.csv"
use_cols =['tpep_pickup_datetime', 'tpep_dropoff_datetime']
df = pd.read_csv(file_path, usecols=use_cols)
original_df = df.copy()
df.to_csv('new_Cabs.csv', index=False)

file_path = "new_Cabs.csv"
df = pd.read_csv(file_path)


Downloading...
From (original): https://drive.google.com/uc?id=1w09SlKhsfw8wkY7Z0z7KbRmPEDekpmT3
From (redirected): https://drive.google.com/uc?id=1w09SlKhsfw8wkY7Z0z7KbRmPEDekpmT3&confirm=t&uuid=e32ae04c-681f-493b-817a-654374df0200
To: /content/CABS.csv
100% 4.06G/4.06G [00:42<00:00, 95.3MB/s]


showing the original dataset, as you can see none of the columns are usable except ['tpep_pickup_datetime', 'tpep_dropoff_datetime']

In [3]:
original_df= pd.read_csv('CABS.csv')
original_df.head(5)

<ipython-input-3-451d2a69261b>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  original_df= pd.read_csv('CABS.csv')


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [4]:
def get_outliers(df, series):
  q1 = series.quantile(0.25)
  q3 = series.quantile(0.75)

  if q1*q3 == 0:
    iqr = abs(2*(q1+q3))
    toprange = iqr
    botrange = -toprange
  else:
    iqr = q3-q1
    toprange = q3 + iqr * 1.5
    botrange = q1 - iqr * 1.5

  outliers_top=df[series > toprange]
  outliers_bot= df[series < botrange]
  outliers = pd.concat([outliers_bot, outliers_top], axis=0)

  return (botrange, toprange, outliers)

In [5]:
# # copying the file to work on a safe file
work_df = df.copy()
print(f"shape = {work_df.shape}")
print(f"Checking for nulls: {work_df.isnull().sum()}")
# #adjusting the data
work_df['tpep_pickup_datetime'] = pd.to_datetime(work_df['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
# extract date and time into separate columns
work_df['tpep_pickup_date'] = work_df['tpep_pickup_datetime'].dt.date
work_df['hour_pickup'] = work_df['tpep_pickup_datetime'].dt.hour
work_df.drop(['tpep_dropoff_datetime'],axis=1,inplace=True)
table = pd.crosstab(index=[work_df['tpep_pickup_date'], work_df['hour_pickup']], columns=['count'])
# # in order to work better on less time per refresh
table.to_csv('final_cabs.csv', index=True)
final_path = 'final_cabs.csv'
final_df = pd.read_csv(final_path)
final_df.set_index(['tpep_pickup_date','hour_pickup'],inplace=True)


shape = (38426000, 2)
Checking for nulls: tpep_pickup_datetime     0
tpep_dropoff_datetime    0
dtype: int64


In [6]:
# #
# # # NOTICE that the numbers represented are given from monday where monday is 0  to sunday where sunday is 6 in day_of_week
# #
final_df['day_of_week']= pd.to_datetime(final_df.index.get_level_values(0).astype(str))
final_df['day_of_week'] = final_df['day_of_week'].dt.weekday
final_df['month_bin'] = pd.to_datetime(final_df.index.get_level_values(0).astype(str))
final_df['month_bin'] = final_df['month_bin'].dt.month
# extract the hour component from the datetime format
#define the bins and labels
bins_days = [0, 6, 12, 18, 23]
labels_days = [0, 1, 2, 3]
bins_months = [1, 4, 8, 12]
labels_months = [1, 0, 2]
# bin the hours and assign labels to each bin
final_df['pickup_bin'] = pd.cut(final_df.index.get_level_values(1), bins=bins_days, labels=labels_days, include_lowest=True)
final_df['month_bin'] = pd.cut(pd.to_datetime((final_df.index.get_level_values(0))).month, bins=bins_months, labels=labels_months, include_lowest=True)
final_df.to_csv('model_cabs.csv',index=True)
# building the model:
# since we do not have a lot of features, we will not do feature selection, and we wish to keep them.
# which means we will use RandomForestRegressor for linear regression
final_df = pd.read_csv("model_cabs.csv")
holidays=['12-31','03-17','07-04','10-31','12-24']
final_df.set_index(['tpep_pickup_date','hour_pickup'],inplace=True)
final_df['holiday']= [1 if date.strftime('%m-%d') in holidays else 0 for date in pd.to_datetime(final_df.index.get_level_values(0))]
print(final_df.columns)

botrange, toprange, outliers = get_outliers(final_df, final_df['count'])
print(toprange)
print(botrange)

print(outliers)


Index(['count', 'day_of_week', 'month_bin', 'pickup_bin', 'holiday'], dtype='object')
12898.625
-4412.375
Empty DataFrame
Columns: [count, day_of_week, month_bin, pickup_bin, holiday]
Index: []


In [7]:

X = final_df.drop('count',axis=1)
y = final_df['count']
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12345)

# Create a Gradient Boosted Regression Trees model
model = RandomForestRegressor(n_estimators=1000, random_state=12345)

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

print(len(y_pred))
# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error:{rmse}")
features = ['day_of_week','month_bin' ,'pickup_bin' ,'holiday']


2609
Mean Squared Error: 1572957.8381473941
Root Mean Squared Error:1254.176159136903


lost the file of the test so i comment it out

In [8]:

# #bin the hours and assign labels to each bin

# new_inputs = pd.read_csv(r'TestDataForStudents.csv')
# bins_days = [0, 6, 12, 18, 23]
# labels_days = [0, 1, 2, 3]
# bins_months = [1, 4, 8, 12]
# labels_months = [1, 0, 2]
# checking_df = pd.DataFrame()
# print(new_inputs.head(5))
# date=[]
# time=[]
# for row in new_inputs.values.tolist():
#   date.append(row[0])
#   time.append(row[1])

# print(time)
# print(date)
# checking_df['tpep_pickup_datetime'] = pd.to_datetime(new_inputs['Dates'])
# checking_df['day_of_week'] = checking_df['tpep_pickup_datetime'].dt.dayofweek
# checking_df['pickup_bin'] = pd.cut(time, bins=bins_days, labels=labels_days, include_lowest=True)
# checking_df['month_bin'] = pd.cut(date, bins=bins_months, labels= labels_months, include_lowest=True)
# checking_df['holiday'] = [1 if date.strftime('%m-%d') in holidays else 0 for date in pd.to_datetime(date)]
# checking_df.set_index(checking_df['tpep_pickup_datetime'],inplace=True)

# predictions = model.predict(checking_df[features])
# print(predictions)

# #uploading submissions
# new_inputs['predictions']=predictions
# new_inputs.to_csv('SubmissionTestDataForStudents.csv')